In [1]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import absolute
from numpy import sqrt
import random
from sklearn import metrics
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import xlsxwriter
import openpyxl
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.manifold import TSNE
from sklearn.linear_model import LinearRegression
from sklearn.cluster import OPTICS
from sklearn.cluster import KMeans
from scipy.stats import ks_2samp
from scipy.stats import wilcoxon
from skbio.diversity import beta_diversity, alpha_diversity
from skbio.stats.ordination import pcoa, pcoa_biplot
from skbio import DistanceMatrix
from dtaidistance import dtw, clustering
from tslearn.clustering import TimeSeriesKMeans
import os
from pandas import Series, ExcelWriter
import scipy.stats as stats
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
import seaborn as sns
import scanpy as sc 
import csv
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')

In [3]:
import sys  
sys.path.insert(0, '../my_packages')
import dimen_reduc
import meta_ord
import paga_res
import sliding_window
import statistic

In [2]:
# 1. Add delta from previous pseudotime to each sample
# 2. find distribution of deltas in menstruation or in non-menstruation.
# 3. Compare between delta's

#### Varaibles

In [6]:
file_path = 'C:/Users/morts/Documents/TAU/Manifold_Project/Data/Serrano_Ravel13_Ravel11_Carter22/ps_res/'

file_name = file_path + 'ps_res_16052023.xlsx'
nn_file_name = file_path + 'nn_res_30052023.xlsx'

sheet_abun = 'abundance'
sheet_meta = 'meta'
sheet_umap = 'umap'
sheet_pcoa = 'pcoa'

#### Load data

In [7]:
meta = pd.read_excel(file_name, sheet_name = sheet_meta, index_col = 0)
nn_meta = pd.read_excel(nn_file_name, sheet_name = sheet_meta, index_col = 0)
print(meta.shape)
print(nn_meta.shape)

(7842, 31)
(514, 35)


In [8]:
meta = nn_meta.copy()

In [9]:
temp_meta = meta[meta['sampleID'].str.contains('UAB')]#meta.loc[~ meta['time'].isnull()]
temp_meta['menst'] = np.where(temp_meta['menst'] >= 1 , 1, 0)
print('Temporal samples shape: ' + str(temp_meta.shape))

Temporal samples shape: (484, 35)


#### Delta to previous sample

In [10]:
temp_meta = temp_meta.sort_values(['subjectID', 'time'])

In [11]:
temp_meta['prev_pseudotime'] = temp_meta.groupby('subjectID')['mt_pseudotime'].shift(periods = 1)
temp_meta['ps_diff'] = temp_meta['mt_pseudotime'] - temp_meta['prev_pseudotime']
temp_meta['ps_diff_abs'] = temp_meta['ps_diff'].abs()

In [22]:
tmp = temp_meta[['time', 'subjectID', 'menst']]
tmp['prev_menst1'] = tmp.groupby('subjectID')['menst'].shift(periods = -1)
tmp['prev_menst2'] = tmp.groupby('subjectID')['menst'].shift(periods = -2)
tmp['prev_menst3'] = tmp.groupby('subjectID')['menst'].shift(periods = -3)
tmp['next_menst1'] = tmp.groupby('subjectID')['menst'].shift(periods = 1)
tmp['next_menst2'] = tmp.groupby('subjectID')['menst'].shift(periods = 2)
tmp['next_menst3'] = tmp.groupby('subjectID')['menst'].shift(periods = 3)
tmp['sum_all'] = tmp.loc[:,tmp.columns.str.contains('menst')].sum(axis = 1)
tmp['sum_all'][tmp['sum_all'] >= 1] = 1

tmp.rename(columns = {'menst':'m_orig'}, inplace = True)
tmp['sum_without'] = tmp.loc[:,tmp.columns.str.contains('menst')].sum(axis = 1)
tmp['sum_without'][tmp['sum_without'] >= 1] = 1

tmp['sum_closer_without'] = tmp.loc[:,tmp.columns.str.contains('|'.join(['1', '2']))].sum(axis = 1)
tmp['sum_closer_without'][tmp['sum_closer_without'] >= 1] = 1

tmp['sum_closer'] = tmp.loc[:,tmp.columns.str.contains('|'.join(['1', '2', 'orig']))].sum(axis = 1)
tmp['sum_closer'][tmp['sum_closer'] >= 1] = 1

In [13]:
temp_meta = temp_meta.join(tmp[['sum_all', 'sum_without', 'sum_closer_without', 'sum_closer']])

In [14]:
temp_meta.groupby('sum_closer').count()

sampleID  subjectID  day  week  time   ph  nugent  menst  subCST  \
sum_closer                                                                     
0.0              252        252  252   252   252  247     146    252     252   
1.0              232        232  232   232   232  229     143    232     232   

            CST  ...  closest_subCST  nugent_bin  ph_bin  BV_amsel_bin  \
sum_closer       ...                                                     
0.0         252  ...             252         146     247           100   
1.0         232  ...             232         143     229            84   

            prev_pseudotime  ps_diff  ps_diff_abs  sum_all  sum_without  \
sum_closer                                                                
0.0                     247      247          247      252          252   
1.0                     225      225          225      232          232   

            sum_closer_without  
sum_closer                      
0.0                        252  
1.0                        232  

[2 rows x 41 columns]

#### find distribution in menstruation and non-menstruation

In [15]:
g = temp_meta.groupby('subjectID')['menst'].sum()
groups = [index for index in g.index if g.loc[index] > 0]
temp_menst_meta = temp_meta.loc[temp_meta['subjectID'].isin(groups)]
display(len(temp_menst_meta.subjectID.unique()))

12

#### Statistics

In [107]:
# temp_menst_meta.menst.unique()
# print(statistic.dist_diff(temp_menst_meta, 'sum_rest', 'ps_diff', False))
# temp_menst_meta.hist(column = 'mt_pseudotime', by = 'sum_rest', bins = 25)

#### Difference in change

In [17]:
def get_wilcox(df, x, y, alt):
    temp_df = df[[x, y]]
    group0 = temp_df[y][temp_df[x] == 0].tolist()
    group1 = temp_df[y][temp_df[x] == 1].tolist()
    
    # Check normality
    norm_pval0 = stats.kstest(group0, 'norm').pvalue
    norm_pval1 = stats.kstest(group1, 'norm').pvalue

    # Check difference between distributions
    dist_pval = ks_2samp(group0, group1).pvalue
    
    print('group0 not normal dist (<0.05): ', str(norm_pval0))
    print('group1 not normal dist (<0.05): ', str(norm_pval1))
    print('not same dist (<0.05): ', str(dist_pval))
    
    ## groups do not come from the same distribution or not normal
    if norm_pval0 < 0.05 or norm_pval1 < 0.05 or dist_pval < 0.05: 
        p_val = stats.mannwhitneyu(group0, group1, alternative = alt)

    ## groups come from the same distribution and normal
    else: 
        p_val = stats.ttest_ind(group0, group1)
    print(p_val)
    
    return p_val[1]

def get_box(df, x, y, alt):
    fig = px.box(df, x = x, y = y, width = 500, height = 350)
    for s in df[x].unique():
        fig.add_annotation(x = s,
                           y = df[df[x] == s][y].max(),
                           text = str(len(df[df[x] == s][x])),
                           yshift = 10,
                           showarrow = False)
#     fig.show()
    
    results = 0
    if len(df[x].unique()) == 2:
        results = get_wilcox(df, x, y, alt)
    
    return fig, results

def get_scatter(df, x, y):
    fig = px.scatter(df, x = x, y = y, trendline = 'ols', width = 600, height = 400)
    fig.show()
    
    results = px.get_trendline_results(fig)
    
    display(results.iloc[0]["px_fit_results"].params)
    display(results.iloc[0]["px_fit_results"].summary())

    return fig, results

def get_plot(df, x, y, nona, func, alt):
    if nona:
        df_nona = df[df[x].notna()]
    else:
        df[x] = df[x].fillna(0)
        df_nona = df
    
    fig, results = func(df_nona, x, y, alt)
    
    return fig, results

In [14]:
# temp_menst_meta[['subjectID', 'menst', 'mt_pseudotime', 'change_menst_status', 'ps_diff_abs']].loc[temp_menst_meta['subjectID'] == 'UAB125']

In [18]:
wilcoxon(temp_menst_meta['sum_closer_without'], temp_menst_meta['ps_diff_abs'])

WilcoxonResult(statistic=23008.0, pvalue=1.77326702094048e-22)

In [19]:
res_lst = {}
for col in ['sum_all', 'sum_without', 'sum_closer_without', 'sum_closer']:
    print('*** ', col, '***')
    fig, res = get_plot(temp_menst_meta.dropna(subset = ['sum_all', 'ps_diff_abs']), col, 'ps_diff', True, get_box, 'greater') 
    
    res_lst[col] = res
    if res < 0.05:
        fig.show()

***  sum_all ***
group0 not normal dist (<0.05):  1.6805504782226417e-37
group1 not normal dist (<0.05):  4.0697949780982795e-36
not same dist (<0.05):  0.14467428207891686
MannwhitneyuResult(statistic=28481.0, pvalue=0.3245035031744802)
***  sum_without ***
group0 not normal dist (<0.05):  5.792069805686765e-38
group1 not normal dist (<0.05):  3.1952587371955633e-35
not same dist (<0.05):  0.1535144939750419
MannwhitneyuResult(statistic=28651.0, pvalue=0.29273515594501387)
***  sum_closer_without ***
group0 not normal dist (<0.05):  4.127312821781161e-40
group1 not normal dist (<0.05):  4.8126051215722134e-33
not same dist (<0.05):  0.27513926711510317
MannwhitneyuResult(statistic=28320.0, pvalue=0.3338278040879261)
***  sum_closer ***
group0 not normal dist (<0.05):  7.21580871694106e-39
group1 not normal dist (<0.05):  3.0901348634595117e-34
not same dist (<0.05):  0.29332759554769494
MannwhitneyuResult(statistic=28251.0, pvalue=0.37705136726064525)


In [21]:
save_cols = ['subjectID', 'CST', 'subCST', 'time', 'day', 'week', 'mt_pseudotime', 'menst', 'ps_diff', 'ps_diff_abs','sum_all', 'sum_without', 'sum_closer_without', 'sum_closer']
dict_menst = {'meta': temp_menst_meta[save_cols]}
menst_path = file_path + 'nn_menst_statistics_res.xlsx'
paga_res.save_excel(menst_path, dict_menst)